![## **LIMPIEZA DE DATOS**](https://th.bing.com/th/id/R.ff326f0dbaeab15371329cc4b3503f02?rik=7oY1UwiGv6n%2frg&riu=http%3a%2f%2fpluspng.com%2fimg-png%2flogo-mondelez-png-markalar-banner-895.png&ehk=DuZgIp4QcYPv%2fDijfYorxucYp9yBN64tinb0vkCqggE%3d&risl=&pid=ImgRaw&r=0)

En este cuaderno se llevará a cabo la creación de columnas clave para la alimentación del tablero dedicado al distribuidor SCN de Mondelez. De esta manera, la creación de estas columnas estará fundamentada y documentada para facilitar la interpretación de los gráficos y resultados presentados.

In [127]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# Configuro pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

Cargamos los datos de SCN obtenidos, los cuales fueron extraídos de la página de Randstad y han sido recopilados y limpiados. Esta base de datos incluye una agrupación de distintos reportes proporcionados por la página de Randstad:

1. **Reporte de Tareas**:
   Este reporte contiene los datos de los locales que han sido efectivamente censados, incluyendo las respuestas correspondientes a la encuesta planificada.

2. **Reporte de Marcas Manuales**:
   Este reporte incluye información de locales core y no core de la zona en estudio.

3. **Reporte de Observaciones**:
   En este reporte se detalla la información de instancias particulares del censo, como locales que rechazaron la encuesta, proporcionaron poca información, estaban cerrados al momento del relevamiento por el censista o cerrados definitivamente.

In [128]:
df = pd.read_excel("SCN.xlsx") 

In [129]:
df

,nombre_alta,Identificador,Direccion_censo,Departamento,Canal,Provincia,Departamentos,Localidad,Coordenadas,lat,long,identificador_censita,nombre_censista_x,Jerarquía nivel 1,Fecha,"¿Realizó alguna acción? Llámese tareas, control de sku, observaciones, etc",Editado,origen,ID_censista,Jerarquia 1 del punto de interés,Tipo de Punto de Venta,Razón Social,CUIT,Papeles?,Cantidad de Cajas(Autoservicio),Cantidad de Góndolas(Autoservicio),Tiene Verdulería(Autoservicio),Tiene Fiambrería(Autoservicio),Heladeras con lacteos(Autoservicio),Heladeras con bebidas(Autoservicio),Cantidad de Exhibidores de primeras marcas(Almacen),Tiene surtido de vinos(Almacen),Tiene Heladera de lácteos(Almacen),Tiene gondola(Maxi Kiosco),"si tenga gondola, cuan llena esta?(Maxi Kiosco)",Tiene exhibidor de chicles/caramelos(Maxi Kiosco),cuan lleno esta?(Maxi Kiosco),No tiene servicios adicionales(Maxi Kiosco),Fotocopias(Maxi Kiosco),RapiPago(Maxi Kiosco),Tramites(Maxi Kiosco),SUBE(Maxi Kiosco),Cuan llena estan las estanterias(Kiosco Ventana),Comercializa chocolates(Vinoteca),Comercializa productos de kiosco(farmacia),Modalidad de Venta(Goloteca/Golsineria),Tabletas(Goloteca/Golsineria),Bombones(Goloteca/Golsineria),Alfajores(Goloteca/Golsineria),Chicles(Goloteca/Golsineria),Caramelos(Goloteca/Golsineria),Galletas(Goloteca/Golsineria),Cantidad de góndolas/estanterías(Dietetica),Llenado de estanterías(Dietetica),Trabaja con?,Distribuidor,Frecuencia de Visita,Frecuencia de entrega,Se acerca un repositor de Mondelez?,Se acerca un repositor de la competencia?,"En caso de que sea otro, indicar con quien",Aplicaciones B2B,MDLZ_1,Arcor_1,BEES,Serenisima,Massalin,arcor_e,georgalos_e,quilmes_e,pepsi_e,cocacola_e,cigarillos_e,mondelez_e,Observaciones,Cantidad_de_heladeras_1,Estado_de_Inmueble_1,Oportunidades,BC(Dietetica),Aguila(Dietetica),CerealMix(Dietetica),Turrón Arcor(Dietetica),Chocolate sin Azúcar Georgalos(Dietetica),Georgalos(Dietetica),Cigarros(Dietetica),Que servicios adicionales tiene,Tipos de observación,Descripción,no_nulos,Latitud,Longitud,PDVID,Nombre,N° Poligono,En zona
0,Kiosco oliva,KIA-947095,Thames 31,CABA,Kiosco Ventana,Buenos Aires,Capital,Villa Crespo,"-34.59925352893666,-58.44834286719561",-345992535289367,-584483428671956,TR-029883,Tobias Rodríguez,NaN,2024-04-03 10:34:29,NaN,NaN,efectivamente censado,TR-029883,Kiosco Ventana,Kiosco Ventana,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mayorista,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,verdadero,verdadero,verdadero,verdadero,verdadero,verdadero,verdadero,NaN,1.0,Ordenado y Limpio,falso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-34.599254,-58.448343,NaN,1.0,C1-P1,SI
1,Kiosco vasquez,KIZ-717844,Av juan bautista,CABA,Maxi Kiosco,Buenos Aires,Capital,Villa crespo,"-34.59845621757849,-58.44969470053912",-345984562175785,-584496947005391,TR-029883,Tobias Rodríguez,NaN,2024-04-03 10:39:08,NaN,NaN,efectivamente censado,TR-029883,Maxi Kiosco,Maxi Kiosco,Vázquez gorriti elizabet,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si,1.00,Si,0.75,falso,verdadero,falso,falso,falso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mayorista,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,verdadero,verdadero,verdadero,verdadero,verdadero,verdadero,verdadero,NaN,3.0,Ordenado y Limpio,falso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-34.598456,-58.449695,NaN,1.0,C1-P1,SI
2,Maxikiosco,MA -726802,Av wagner,CABA,Maxi Kiosco,Buenos Aires,Capital,Villa crespo,"-34.60068447526336,-58.44661116600036",-346006844752634,-584466111660004,TR-029883,Tobias Rodríguez,NaN,2024-04-03 10:56:37,NaN,NaN,efectivamente censado,TR-029883,Maxi Kiosco,Maxi Kiosco,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si,0.50,Si,1.00,falso,verdadero,falso,falso,verdadero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mayorista,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,verdadero,verdadero,verdadero,verdadero,verdadero,verdadero,verdadero,NaN,3.0,,falso,NaN,N

**Paso N°1** 

procederemos a identificar a los clientes utilizando el siguiente marco teórico:

En la columna 'Trabaja con?', filtraremos los registros donde se indique 'Distribuidora'. Además, en la columna 'mondelez_e', que indica la presencia de productos Mondelez en el punto de interés, consideraremos que si el local trabaja con una distribuidora y tiene productos Mondelez, aplicaremos la premisa de que es cliente. Para determinar si un local es cliente o una posible oportunidad, necesitamos información proporcionada por el reporte de tareas, es decir, de los locales "efectivamente censados". Dado que el reporte de entrada y salida manual y el reporte de observaciones solo proporcionan información genérica y sustancial de los locales, no disponemos de una columna que nos permita discernir si un punto de interés es o no cliente. Por lo tanto, para determinar las oportunidades, nos centraremos exclusivamente en los locales efectivamente censados.

In [131]:
canales_permitidos = [
    'Kiosco Ventana', 'Maxi Kiosco', 'Autoservicio', 'Kiosco Cadena',
    'Dietética', 'Almacén', 'Goloteca', 'Golsinería', 'Vinoteca'
]

# Aplicar las condiciones la columna 'cliente'
df['cliente'] = (
    (df['Trabaja con?'] == 'Distribuidor') & 
    (df['mondelez_e'] == 'verdadero') & # tiene productos mondelez
    (df['Canal'].isin(canales_permitidos)) & 
    (df['En zona'] == 'SI') & 
    (df['origen'] == 'efectivamente censado')
).astype(int)

In [132]:
df['cliente'].value_counts()

cliente
0    1184
1     761
Name: count, dtype: int64

**Paso N°2**

Dado que no contamos con una columna que sirva como punto de comparación entre la cartera de clientes y los datos obtenidos del censo, se realizó una comparación utilizando los datos de dirección tanto de los puntos de venta de la cartera de clientes como de los puntos de venta censados. Se consideraron como clientes aquellos que comparten una dirección exacta. Es importante destacar que la identificación de los locales encontrados en el censo, ya sea que hayan sido censados o no, de la cartera de clientes solo es completamente beneficiosa si han sido censados. Si un local no ha sido censado, no disponemos de información para determinar si representa una oportunidad o no.

A continuación, procederemos a cargar una base de datos que compara las direcciones entre la cartera de clientes y los locales censados. Se fundamenta en que si un punto de venta de la cartera de clientes y un punto de venta encontrado en alguno de los reportes proporcionados por Randstad comparten exactamente la misma dirección, entonces el punto de venta censado se considera cliente.

In [133]:
df_direccion = pd.read_excel("cleintes, altas, de uno a muchos, radio 99m.xlsx",sheet_name="hoja1")

In [134]:
df_direccion

,nombre_alt,Identifica,Direccion_,Departamen,Canal,Provincia,Departam_1,Localidad,Coordenada,lat,long,identifi_1,nombre_cen,Jerarquía,Fecha,¿Realizó,Editado,origen,ID_censist,Jerarquia,Tipo de Pu,Razón Soc,CUIT,Papeles?,Cantidad d,Cantidad_1,Tiene Verd,Tiene Fiam,Heladeras,Heladera_1,Cantidad_2,Tiene surt,Tiene Hela,Tiene gond,si tenga g,Tiene exhi,cuan lleno,No tiene s,Fotocopias,RapiPago(M,Tramites(M,SUBE(Maxi,Cuan llena,Comerciali,Comercia_1,Modalidad,Tabletas(G,Bombones(G,Alfajores(,Chicles(Go,Caramelos(,Galletas(G,Cantidad_3,Llenado de,Trabaja co,Distribuid,Frecuencia,Frecuenc_1,Se acerca,Se acerc_1,En caso de,Aplicacion,MDLZ_1,Arcor_1,BEES,Serenisima,Massalin,arcor_e,georgalos_,quilmes_e,pepsi_e,cocacola_e,cigarillos,mondelez_e,Observacio,Cantidad_d,Estado_de_,Oportunida,BC(Dieteti,Aguila(Die,CerealMix(,Turrón Ar,Chocolate,Georgalos(,Cigarros(D,Que servic,Tipos de o,Descripci�,no_nulos,Latitud,Longitud,MONTH,TDID,NAME,SALESSTRUC,PDVID,PDVCODE,PDVDESCRIP,CHANNEL,LATITUDE,LONGITUDE,STRATEGIC,CITY,STATEDESCR,DEPARTMENT,ADDRESS,SUM(DARWIN,sin acentos,pos_n,calle_address,numero_address,pos_n_address,calle_dire,numero_dire,proximidad,coincide calle y numero,dire_ok,adress_ok,coincide calle,coincide numero,coincide numero y calle,diferencia numero
0,Kiosco oliva,KIA-947095,Thames 31,Buenos Aires,Kiosco Ventana,Buenos Aires,Belgrano,Villa crespo,"-34.59925352893666,-58.44834286719561",-34.599254,-58.448343,TR-029883,Tobias Rodríguez,NaN,2024-04-03 10:34:29,NaN,NaN,efectivamente censado,TR-029883,Kiosco Ventana,Kiosco Ventana,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mayorista,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,Ordenado y Limpio,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-34,59925353","-58,44834287",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Thames,31.0,NaN,NaN,Thames,NaN,NO,NO,NO,1000000
1,Maxikiosco 24 hs,MAS-129416,Av Pueyrredón 1800,Buenos Aires,Maxi Kiosco,Buenos Aires,Belgrano,Villa crespo,"-34.60087600217769,-58.4511387348175",-34.600876,-58.451139,TR-029883,Tobias Rodríguez,NaN,2024-04-03 09:59:43,NaN,NaN,efectivamente censado,TR-029883,Maxi Kiosco,Maxi Kiosco,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si,75%,Si,75%,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mayorista,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2.0,En condiciones aceptables,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-34,600876","-58,45113873",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Av Pueyrredón,1800.0,NaN,NaN,Pueyrredón,NaN,NO,NO,NO,1000000
2,Antonino,ANO-313648,Honorio Pueyrredon 1000,CABA,Restaurant,Buenos Aires,Capital,Caballito,"-34.60795749632217,-58.4448754414916",-34.607957,-58.444875,RZ-623877,Ricardo Zorkendorfer,NaN,2024-04-03 10:02:22,NaN,NaN,efectivamente censado,RZ-623877,Restaurant,Restaurant,El encargado no autorizo informacion,El encargado no autorizo informacion,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-34,6079575","-58,44487544",202402.0,441.0,SCN DISTRIBUCION SA.,CAPITAL,211035967.0,7598.0,LIN JINGJING,Autoservicio,-34.608626,-58.444471,2.0,C.A.B.A.,Capital Federal,CABA - Villa Crespo,Av. Dr. Honorio PueyrredÃ³n 1023,21.062417,Av. Dr. Honorio Pueyrredon 1023,28.0,Av. Dr. Honorio Pueyrredon,1023.0,20,Honorio Pueyrredon,1000.0,distinta calle o mal escrita,NaN,Honorio Pueyredon,Honorio Pueyredon,SI,NO,NO,-23
3,Antonino,ANO-313648,Honorio Pueyrredon 1000,CABA,Restaurant,Buenos Aires,Capital,Caballito,"-34.60795749632217,-58.4448754414916",-34.607957,-58.444875,RZ-623877,Ricardo Zorkendorfer,NaN,2024-04-03 10:02:22,NaN,NaN,efectiva

In [135]:
# Filtramos los valores donde 'coincide numero y calle diferencia' es igual a 'SI'
df_direccion = df_direccion[df_direccion['coincide numero y calle'] == 'SI']

# Obtenemos los valores únicos de la columna 'identificador' y los guardamos en una lista
valores_unicos = df_direccion['Identifica'].unique().tolist()

# Mostramos la lista resultante
print(valores_unicos)


['FIS-275747', 'ELI-216800', 'MAA-428895', 'MCC-252611', 'PU4-976820', 'KIA-106488', 'AUS-231390', 'SUO-490120', 'KIP-381391', 'MA -284117', 'PU4-194245', 'DIN-421478', 'LA -363782', 'SEÑ-241572', 'LOS-660004', 'OAS-684082', 'KIO-510252', 'KI0-991856', 'KI -164736', 'AU -071508', 'KI -068018', 'KI -767068', 'MA -654632', 'KI -295893', 'DRE-670581', 'KIO-071261', 'PAO-342789', 'SR -414844', 'WEG-023501', 'CA -574637', 'KI -088130', 'SU -503139', 'KIA-754642', 'KI -643112', 'M2T-967850', 'AL -905466', 'MA -376327', 'KIK-510337', 'HOE-233053', 'CAI-585284', 'PAI-971934', 'KIO-015667', 'LA5-469280', 'KIO-677295', 'AU -623341', 'MA -713794', 'SUE-384551', 'KIO-361485', 'MA -177912', 'MA -813719', 'M 4-491046', 'SAR-475935', 'NAL-123063', 'SKO-901381', 'AU0-005623', 'KIO-291268', 'TEY-339181', 'MAH-112252', 'AUL-812305', 'GOT-354635', 'MAO-525628', 'AU -851484', 'KIA-230945', 'GIO-309338', 'CAO-202823', 'ELI-787599', 'MAO-437818', 'PEA-048489', 'LOA-742864', 'LOS-067191', 'KIO-797648', 'KIO-

In [136]:
#sumamos datos de clientes obtenidos por la direccion. 
for valor in valores_unicos:
    df.loc[
        (df['Identificador'] == valor) & 
        (df['cliente'] == 0) & 
        (
            (df['Trabaja con?'] == 'Distribuidor') & 
            (df['mondelez_e'] == 'verdadero') & 
            (df['Canal'].isin(canales_permitidos)) & 
            (df['En zona'] == 'SI') & 
            (df['origen'] == 'efectivamente censado')
        ), 
        'cliente'
    ] = 1

In [137]:
df['cliente'].value_counts()

cliente
0    1184
1     761
Name: count, dtype: int64

**Oportunidades**

Las oportunidades se segmentan entre los locales que no son considerados clientes. Estos se consideran oportunidades ya que no reciben suministros a través de los distribuidores de Mondelez. Además, se realizará un conteo de los puntos de interés que no son clientes de Mondelez para determinar cuántos de ellos tienen productos de la competencia. Se consideran competencia las marcas Arcor y Georgalos.

In [143]:
# creación de la columna oportunidades 
df['oportunidad_1'] = (
    (df['En zona'] == 'SI') &
    (df['origen'] == 'efectivamente censado') &
    (df['Trabaja con?'].isin(['Mayorista', 'Otro'])) &
    (df['cliente'] == 0) &
    (df['Canal'].isin(canales_permitidos))
).astype(int)


In [144]:
df['oportunidad_1'].value_counts()

oportunidad_1
0    1619
1     326
Name: count, dtype: int64

In [146]:
# Crear la columna 'No es cliente y no tiene productos mondelez'
df['No es cliente y tiene productos de la competencia'] = (
    (df['cliente'] == 0) & 
    ((df['arcor_e'] == 'verdadero') | (df['georgalos_e'] == 'verdadero'))
).astype(int)

In [147]:
df['No es cliente y tiene productos de la competencia'].value_counts()

No es cliente y tiene productos de la competencia
0    1484
1     461
Name: count, dtype: int64

Guardamos los cambios realizados y exportamos la base a un XLSX

In [148]:
df.to_excel("SCN_tablero.xlsx", index= False)